# Build a cuisine recommender

In [113]:
import pandas as pd 
df = pd.read_csv("../data/genres_v2.csv")
df.head()

/var/folders/dk/n6dfn2b91tjgclxq8nc4pw7r0000gn/T/ipykernel_51170/2224900341.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/genres_v2.csv")


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,Unnamed: 0,title
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,...,2Vc6NJ9PW9gD9q343XFRKx,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,https://api.spotify.com/v1/tracks/2Vc6NJ9PW9gD...,https://api.spotify.com/v1/audio-analysis/2Vc6...,124539,4,Dark Trap,Mercury: Retrograde,NaN,NaN
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,...,7pgJBLVz5VmnL7uGHmRj6p,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,https://api.spotify.com/v1/tracks/7pgJBLVz5Vmn...,https://api.spotify.com/v1/audio-analysis/7pgJ...,224427,4,Dark Trap,Pathology,NaN,NaN
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,...,0vSWgAlfpye0WCGeNmuNhy,spotify:track:0vSWgAlfpye0WCGeNmuNhy,https://api.spotify.com/v1/tracks/0vSWgAlfpye0...,https://api.spotify.com/v1/audio-analysis/0vSW...,98821,4,Dark Trap,Symbiote,NaN,NaN
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,...,0VSXnJqQkwuH2ei1nOQ1nu,spotify:track:0VSXnJqQkwuH2ei1nOQ1nu,https://api.spotify.com/v1/tracks/0VSXnJqQkwuH...,https://api.spotify.com/v1/audio-analysis/0VSX...,123661,3,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote),NaN,NaN
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,...,4jCeguq9rMTlbMmPHuO7S3,spotify:track:4jCeguq9rMTlbMmPHuO7S3,https://api.spotify.com/v1/tracks/4jCeguq9rMTl...,https://api.spotify.com/v1/audio-analysis/4jCe...,123298,4,Dark Trap,Venom,NaN,NaN


In [114]:
y = df["genre"]
X = df.iloc[:,0:9]
X.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660


In [115]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [117]:
model = RandomForestClassifier(n_estimators=10)
model.fit(X_train,y_train)


RandomForestClassifier(n_estimators=10)

In [118]:
y_pred = model.predict(X_test)

In [119]:
print(classification_report(y_test,y_pred))

                 precision    recall  f1-score   support

      Dark Trap       0.31      0.33      0.32      1322
            Emo       0.53      0.61      0.57       486
         Hiphop       0.21      0.21      0.21       869
            Pop       0.13      0.09      0.11       129
            Rap       0.10      0.08      0.08       559
            RnB       0.30      0.24      0.27       636
     Trap Metal       0.18      0.12      0.14       606
Underground Rap       0.33      0.39      0.36      1768
            dnb       0.59      0.62      0.60       913
      hardstyle       0.62      0.67      0.64       893
      psytrance       0.66      0.71      0.68       880
      techhouse       0.75      0.79      0.77       879
         techno       0.71      0.66      0.68       888
         trance       0.64      0.57      0.60       915
           trap       0.72      0.67      0.70       949

       accuracy                           0.48     12692
      macro avg       0.45   

In [120]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 380]))]
options = {id(model): {'nocl': True, 'zipmap': False}}
onx = convert_sklearn(model, initial_types=initial_type, options=options)
with open("./model.onnx", "wb") as f:
    f.write(onx.SerializeToString())


